# Imports

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Import essays

In [3]:
import pandas as pd

# Load the dataset
file_path = 'ielts_writing_dataset.csv'  # Make sure the path is correct
dataset = pd.read_csv(file_path)

# Select only the 'Essay' and 'Overall' columns
essays = dataset['Essay']
scores = dataset['Overall']


# Data Preprocessing

In [4]:
# Parameters
vocab_size = 10000  # Adjust as needed
max_length = 300    # Adjust based on your essay length
padding_type = 'post'
trunc_type = 'post'

# Tokenizing the essays
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(essays)
word_index = tokenizer.word_index

# Converting essays to sequences
sequences = tokenizer.texts_to_sequences(essays)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Build

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Single output node for regression
])

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 64)           640000    
                                                                 
 bidirectional (Bidirection  (None, 128)               66048     
 al)                                                             
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 714369 (2.73 MB)
Trainable params: 714369 (2.73 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Training

In [8]:
# Convert scores to numpy array
scores = np.array(scores)

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
train_padded, test_padded, train_scores, test_scores = train_test_split(padded, scores, test_size=0.2)

# Train the model
num_epochs = 10  # Adjust as needed
model.fit(train_padded, train_scores, epochs=num_epochs, validation_data=(test_padded, test_scores))


Epoch 1/10
36/36 [==============================] - 6s 129ms/step - loss: 16.5498 - mean_absolute_error: 3.1732 - val_loss: 1.2628 - val_mean_absolute_error: 0.8797
Epoch 2/10
36/36 [==============================] - 4s 116ms/step - loss: 1.2187 - mean_absolute_error: 0.8875 - val_loss: 1.0994 - val_mean_absolute_error: 0.8251
Epoch 3/10
36/36 [==============================] - 4s 116ms/step - loss: 1.1152 - mean_absolute_error: 0.8417 - val_loss: 1.0731 - val_mean_absolute_error: 0.8164
Epoch 4/10
36/36 [==============================] - 4s 121ms/step - loss: 1.0582 - mean_absolute_error: 0.8183 - val_loss: 1.0889 - val_mean_absolute_error: 0.8349
Epoch 5/10
36/36 [==============================] - 4s 121ms/step - loss: 0.9517 - mean_absolute_error: 0.7750 - val_loss: 0.9777 - val_mean_absolute_error: 0.7850
Epoch 6/10
36/36 [==============================] - 5s 138ms/step - loss: 0.6653 - mean_absolute_error: 0.6319 - val_loss: 0.9676 - val_mean_absolute_error: 0.7699
Epoch 7/10
36/3

# Evaluation

In [9]:
loss, accuracy = model.evaluate(test_padded, test_scores)
print(f'Loss: {loss}, Accuracy: {accuracy}')

9/9 [==============================] - 0s 35ms/step - loss: 1.0347 - mean_absolute_error: 0.7541
Loss: 1.0347354412078857, Accuracy: 0.7541486620903015
